In [1]:
!pip install groq  # Install official Groq Python client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.5 MB/s eta 0:00:00


In [2]:
import os
os.environ["GROQ_API_KEY"] = "groq api key"

In [3]:
import os
from groq import Groq

# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Define a simple conversation message
messages = [
    {"role": "user", "content": "Explain the importance of fast language models"}
]

# Call Groq API for chat completion
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=messages
)

# Print the assistant reply
print("Groq API reply:\n", response.choices[0].message.content)


Groq API reply:
 Fast language models have become increasingly important in recent years due to their wide range of applications and the growing demand for efficient and scalable natural language processing (NLP) systems. Here are some reasons why fast language models are crucial:

1. **Improved User Experience**: Fast language models enable faster inference and response times, which is essential for applications that require real-time or near-real-time interactions, such as:
	* Virtual assistants (e.g., Siri, Alexa, Google Assistant)
	* Chatbots and customer service platforms
	* Language translation systems
	* Sentiment analysis and opinion mining tools
2. **Scalability**: As the amount of text data grows exponentially, fast language models can handle larger volumes of data and scale more efficiently, making them suitable for:
	* Large-scale text analysis and mining
	* Social media monitoring and analysis
	* Web search and information retrieval
3. **Cost-Effectiveness**: Fast language

In [8]:
import os
from groq import Groq

# Initialize Groq client once
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def truncate_by_turns(history, n):
    return history[-n:]

def truncate_by_char_limit(history, char_limit):
    total_chars = 0
    truncated = []
    for msg in reversed(history):
        size = len(msg['content'])
        if total_chars + size > char_limit:
            break
        truncated.append(msg)
        total_chars += size
    return list(reversed(truncated))

class ConversationManager:
    def __init__(self, summarize_every_k=3):
        self.history = []
        self.run_count = 0
        self.k = summarize_every_k

    def add_user_message(self, text):
        self.history.append({"role": "user", "content": text})

    def add_assistant_message(self, text):
        self.history.append({"role": "assistant", "content": text})

    def get_conversation_text(self):
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.history])

    def summarize_history(self):
        prompt = f"Please provide a concise summary of this conversation:\n{self.get_conversation_text()}"
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": prompt}]
        )
        summary = response.choices[0].message.content.strip()
        return summary

    def generate_reply(self, trunc_type=None, trunc_limit=None):
        self.run_count += 1

        # Truncate if specified
        history_to_send = self.history
        if trunc_type == "turns" and trunc_limit:
            history_to_send = truncate_by_turns(self.history, trunc_limit)
        elif trunc_type == "chars" and trunc_limit:
            history_to_send = truncate_by_char_limit(self.history, trunc_limit)

        # Call Groq API for assistant reply
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=history_to_send
        )
        reply = response.choices[0].message.content
        self.add_assistant_message(reply)

        # Perform summarization every k-th run
        if self.run_count % self.k == 0:
            summary = self.summarize_history()
            # Replace conversation with summary as a single assistant message
            self.history = [{"role": "assistant", "content": f"[Summary]\n{summary}"}]

        return reply

    def print_history(self):
        print("Conversation History:")
        for msg in self.history:
            print(f"{msg['role']}: {msg['content']}")


conv_manager = ConversationManager()

# Add multiple user messages to simulate conversation
conv_manager.add_user_message("Hello, can you tell me about Groq API?")
reply = conv_manager.generate_reply()
print("Assistant reply:", reply)

conv_manager.add_user_message("What are some best practices for using Groq API?")
reply = conv_manager.generate_reply(trunc_type="turns", trunc_limit=3)  # Keep last 3 turns only
print("Assistant reply with truncation by turns:", reply)

conv_manager.add_user_message("How can I securely store API keys when using Groq in my projects?")
reply = conv_manager.generate_reply(trunc_type="chars", trunc_limit=500)  # Limit message content to 500 chars
print("Assistant reply with truncation by chars:", reply)

# Print all conversation history
conv_manager.print_history()

Assistant reply: Groq API! That's an interesting one.

Groq is a cloud-based API platform that provides access to a range of AI models, primarily focused on natural language processing (NLP) and computer vision. The Groq API allows developers to integrate these AI capabilities into their applications, making it easier to build intelligent and automated systems.

Here are some key features and facts about the Groq API:

**Key Features:**

1. **AI Model Library:** Groq offers a library of pre-trained AI models for various tasks, such as text classification, sentiment analysis, entity extraction, and image recognition.
2. **API-driven:** The Groq API provides a simple and RESTful interface to access these AI models, allowing developers to integrate them into their applications.
3. **Scalability:** Groq's cloud-based infrastructure ensures scalability and high availability, making it suitable for applications with high traffic or large data volumes.

**Supported AI Models:**

1. **Natural 

In [12]:
user_info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "string"} # Changed from "integer" to "string"
    },
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False
}

In [15]:
def extract_user_info(text):
    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": text}],
        functions=[{
            "name": "extract_user_info",
            "description": "Extract user name, email, phone, location, and age from chat",
            "parameters": user_info_schema
        }],
        function_call={"name": "extract_user_info"}
    )
    # The API returns function call data as JSON string
    func_call = response.choices[0].message.function_call
    extracted_json_str = func_call.arguments if func_call and hasattr(func_call, 'arguments') else "{}"
    import json
    try:
        extracted_data = json.loads(extracted_json_str)
    except json.JSONDecodeError:
        extracted_data = {}
    return extracted_data

In [16]:
samples = [
    "Hi, I'm John Doe, my email is john@example.com, phone 555-1234, living in New York, and I am 30 years old.",
    "Hello, my name is Alice, email alice@example.com, phone number 123-456-7890, based in London. Age 28.",
    "This is Bob Smith. Contact: bobsmith@email.com, phone 9876543210. Location: Toronto. Age: 35."
]

for i, sample in enumerate(samples, 1):
    info = extract_user_info(sample)
    print(f"Sample {i} extracted info:", info)

Sample 1 extracted info: {'age': '30', 'email': 'john@example.com', 'location': 'New York', 'name': 'John Doe', 'phone': '555-1234'}
Sample 2 extracted info: {'age': '28', 'email': 'alice@example.com', 'location': 'London', 'name': 'Alice', 'phone': '123-456-7890'}
Sample 3 extracted info: {'age': '35', 'email': 'bobsmith@email.com', 'location': 'Toronto', 'name': 'Bob Smith', 'phone': '9876543210'}


In [23]:
def validate_extracted_info(data):
    required_fields = ['name', 'email', 'phone', 'location', 'age']
    validated = {}
    for field in required_fields:
        value = data.get(field)
        if not value:
            validated[field] = None
        else:
            if field == 'age':
                try:
                    validated[field] = int(value)
                except:
                    validated[field] = None
            else:
                validated[field] = value
    return validated


In [24]:
for i, sample in enumerate(samples, 1):
    raw_data = extract_user_info(sample)
    clean_data = validate_extracted_info(raw_data)
    print(f"Sample {i} cleaned info:", clean_data)

Sample 1 cleaned info: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '555-1234', 'location': 'New York', 'age': 30}
Sample 2 cleaned info: {'name': 'Alice', 'email': 'alice@example.com', 'phone': '123-456-7890', 'location': 'London', 'age': 28}
Sample 3 cleaned info: {'name': 'Bob Smith', 'email': 'bobsmith@email.com', 'phone': '9876543210', 'location': 'Toronto', 'age': 35}


Okay, here's a brief summary of the steps you've taken in this notebook:

Installed the Groq library: You started by installing the necessary Python library for interacting with the Groq API using !pip install groq. Set up the Groq API Key: You then set your Groq API key as an environment variable using os.environ["GROQ_API_KEY"] = "..." for secure access. Used Groq API for a simple chat completion: You initialized the Groq client and made a basic chat completion call to understand the importance of fast language models. Developed a Conversation Manager: You created a ConversationManager class to handle conversation history, including methods for adding user and assistant messages, retrieving conversation text, summarizing history using the Groq API, and generating replies with optional truncation. You tested this manager with a few turns of conversation. Defined a JSON Schema for User Information: You defined a schema (user_info_schema) to specify the structure and data types for extracting user information (name, email, phone, location, age). Initially, you set 'age' as an integer, but later corrected it to 'string' based on the model's output. Created a Function for Information Extraction: You wrote an extract_user_info function that uses the Groq API with the defined schema and function calling capabilities to extract user information from a given text. You also handled potential errors during JSON parsing. Implemented a Validation Function: You created a validate_extracted_info function to clean and validate the extracted data, specifically converting the 'age' back to an integer if possible and setting fields to None if missing. Tested Extraction and Validation: You tested the extract_user_info and validate_extracted_info functions on a list of sample text inputs (samples), demonstrating how to extract and clean the user information from each sample. In essence, you've set up the Groq API, explored its basic chat capabilities, built a system for managing conversational history with summarization, and developed a process for extracting and validating structured information from text using Groq's function calling feature.

